<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
addprocs()

8-element Array{Int64,1}:
 2
 3
 4
 5
 6
 7
 8
 9

In [2]:
push!(LOAD_PATH, ".")
@everywhere using RobustOptimization

In [5]:
ambiguity = "wasserstein"

N = 62
M = 2000
ϵ = 0.02

A = rand(N,M)
w = 2*rand(N)-1
X = rand(M)
D = A*X
Y = D+w;
Z = hcat(A,Y);

In [6]:
if ambiguity == "wasserstein"
    sample = 100
else
    sample = 100
end 

robustModel = RobustModel(N, M, ϵ, ambiguity, LinearRegression())
α = 1/norm(robustModel.descent_direction)
# α = 0.1
projParams = ProjParams(Int(1e6), 1e-5, sample, para_proj=Sequential(), para_inter=Sequential())
optParams = OptParams(Int(3000), 1e-6, α, verbosity=10);

x0 = rand(size(robustModel.descent_direction));

In [7]:
toto = 60

60

In [16]:
robustModel = RobustModel(N, M, ϵ, ambiguity, LinearRegression())
projParams = ProjParams(Int(70), 1e-5, toto, para_proj=Sequential(), para_inter=Sequential())

RobustOptimization.ProjParams(70, 1.0e-5, 60, RobustOptimization.Sequential(), RobustOptimization.Sequential())

In [17]:
srand(2)
@time algo_proj_bis(x0, Z, robustModel, projParams)

 31.735156 seconds (108.50 k allocations: 222.697 MiB, 0.07% gc time)


([0.617061, 0.0772392, 0.93016, 0.0859882, 0.915367, 0.0236865, 0.0741329, 0.398724, 0.896082, 0.339468  …  0.6434, 0.669016, 0.360005, 0.927408, 0.520115, 0.191397, 0.569706, 0.272882, 0.426261, 0.812725], 0.0, 68)

In [14]:
srand(2)
@time x,d = alg(x0, Z, robustModel, projParams)
print(d)

 29.991655 seconds (291.74 k allocations: 232.655 MiB, 0.08% gc time)
0.0

In [18]:
srand(2)
@time algo_proj(x0, Z, robustModel, projParams)

 61.880163 seconds (24.84 M allocations: 248.237 GiB, 18.76% gc time)


([0.617061, 0.0772392, 0.93016, 0.0859882, 0.915367, 0.0236865, 0.0741329, 0.398724, 0.896082, 0.339468  …  0.6434, 0.669016, 0.360005, 0.927408, 0.520115, 0.191397, 0.569706, 0.272882, 0.426261, 0.812725], 0.0, 68)

In [15]:
srand(2)
@time x,d = alg_tot(x0, Z, robustModel, projParams)
print(d)

 33.708494 seconds (21.35 k allocations: 1.848 MiB)
0.0011247707375904173

In [13]:
@everywhere function ter_proj(I::Array{Int,1},
                    xn::Array{Float64,1},
                    data::Array{Float64,2},
                    robustModel::RobustModel,
                    ω::Float64)
    tab = zeros(length(xn)+2)
    for k in 1:length(I)
        i = I[k]
        name = typeof_constraint(i, robustModel)
        subg = subgradient(xn, data, robustModel.regressionModel, i, name)
        pin = proj_pi(x->fconstraint(x, data, robustModel.regressionModel, i, name), subg, xn, i, data, robustModel.regressionModel)
        aux = max(0, fconstraint(xn, data, robustModel.regressionModel, i, name))
        tab = tab + vcat(ω*pin, ω*dot((pin-xn),(pin-xn)), aux)
    end
    return tab 
end

function alg(x0::Array{Float64,1},
                    data::Array{Float64,2},
                    robustModel::RobustModel,
                    projParams::ProjParams)

    dist_mem::Float64 = projParams.precision+1
    xn::Array{Float64,1} = copy(x0)
    iter::Int = 0
    while (iter < Int(round(length(robustModel.I0)/projParams.sample))+1) || ((dist_mem > projParams.precision) && (iter < projParams.ITER_MAX))
#         println("iter = ", iter, " dm = ", dist_mem)
        iter = iter +1
        I = StatsBase.sample(robustModel.I0, projParams.sample, replace=false, ordered=true)
        push!(I, length(robustModel.I0)+1)
        ω = 1/(length(I))
        im, IM = dispatch_index(length(I),nworkers())
        res = @parallel (+) for k in 1:nworkers()
            ter_proj(I[im[k]:IM[k]], xn, data,robustModel,ω)
        end
#         res::Array{Float64,1} = @parallel (+) for k in 1:length(I)
#             aux_proj(I[k], xn, data, robustModel, ω[k])
#         end
        dist_mem = res[end]/length(I)
        un::Array{Float64,1} = xn-res[1:end-2]
        Ln::Float64 = res[end-1]/(un'*un)
        if (dist_mem <= 0.0) || (dot(un,un) == 0)
            Ln = 1
            # dist_mem = (xn1-xn)'*(xn1-xn)
            I = StatsBase.sample(robustModel.I0, projParams.sample, replace=false, ordered=true)
            dist_mem = x_in_inter(xn, data, robustModel, I, projParams.para_inter)
        end
        xn = QR(x0,xn,xn - Ln*un)
    end
    return xn, dist_mem, iter
end

@everywhere function qua_proj(I::UnitRange{Int},
                    xn::Array{Float64,1},
                    data::Array{Float64,2},
                    robustModel::RobustModel,
                    ω::Float64)
    
    tab = zeros(length(xn)+2)
    for i in I
        name = typeof_constraint(i, robustModel)
        subg = subgradient(xn, data, robustModel.regressionModel, i, name)
        pin = proj_pi(x->fconstraint(x, data, robustModel.regressionModel, i, name), subg, xn, i, data, robustModel.regressionModel)
        aux = max(0, fconstraint(xn, data, robustModel.regressionModel, i, name))
        tab = tab + vcat(ω*pin, ω*dot((pin-xn),(pin-xn)), ω*aux)
    end
    return tab 
end

function alg_tot(x0::Array{Float64,1},
                    data::Array{Float64,2},
                    robustModel::RobustModel,
                    projParams::ProjParams)

    xn::Array{Float64,1} = copy(x0)
    im, IM = dispatch_index(length(robustModel.I0)+1,nworkers())
    I = 1:(length(robustModel.I0)+1)
    res = @parallel (+) for k in 1:nworkers()
        qua_proj(I[im[k]:IM[k]], xn, data,robustModel,1/length(I))
    end
    un::Array{Float64,1} = xn-res[1:end-2]
    xn = QR(x0,xn,xn - res[end-1]/(un'*un)*un)
    return xn, res[end]/length(robustModel.I0)
end

alg_tot (generic function with 1 method)

In [ ]:
using BenchmarkTools

In [ ]:
# push!(LOAD_PATH, ".")
# using RobustOptimization
# ambiguity = "wasserstein"

# N = 100
# ϵ = 0.02

# a = 1*rand()-0.5
# b = 1*rand()-0.5
# w = 2*rand(N)-1
# X = collect(linspace(0,10,N))
# Y = a+b*X+w;
# Z = SharedArray{Float64}(N, 3)
# Z = vcat(hcat(ones(length(X)),X,Y));

# if ambiguity == "KLdivergence"
#     sample = 4
# else
#     sample = 4
# end 

# robustModel = RobustModel(N, 2, ϵ, ambiguity, LinearRegression())
# α = 1/norm(robustModel.descent_direction)
# # α = 0.005
# projParams = ProjParams(Int(1e6), 1e-5, sample, para_proj=Sequential(), para_inter=Sequential())
# optParams = OptParams(Int(3000), 1e-6, α, verbosity=10);
    

# x0 = rand(size(robustModel.descent_direction));

In [ ]:
# x0 = initialize(Z, robustModel, robustModel.name)
x0 = rand(size(robustModel.descent_direction))
xalg, yalg, dm2, mem, mini = run_algo(x0, Z, robustModel, optParams, projParams);

In [ ]:
using PyPlot
plot(dm2)

In [ ]:
using BenchmarkTools
toto = 1000

In [ ]:
srand(10)
projParams = ProjParams(Int(1), 1e-5, toto, para_proj=Parallel(), para_inter=Parallel())
@time algo_proj(x0,Z,robustModel,projParams)
srand(10)
projParams = ProjParams(Int(1), 1e-5, toto, para_proj=Parallel(), para_inter=Sequential())
@time algo_proj(x0,Z,robustModel,projParams)
srand(10)
projParams = ProjParams(Int(1), 1e-5, toto, para_proj=Sequential(), para_inter=Parallel())
@time algo_proj(x0,Z,robustModel,projParams)
srand(10)
projParams = ProjParams(Int(1), 1e-5, toto, para_proj=Sequential(), para_inter=Sequential())
@time algo_proj(x0,Z,robustModel,projParams)

In [ ]:
srand(10)
@time proj_In(x0,Z,robustModel,collect(1:toto),ones(toto)/length(toto),Sequential())
srand(10)
@time proj_In(x0,Z,robustModel,collect(1:toto),ones(toto)/length(toto),Parallel())

In [14]:
function proj_In(xn::Array{Float64,1},
                  data::Array{Float64,2},
                  robustModel::RobustModel,
                  I::Array{Int64,1},
                  ω::Array{Float64,1},
                  para::Sequential)

    n = length(I)
    pin::Array{Float64,1} = zeros(length(xn))
    sum_op = zeros(length(xn))
    Ln = 0
    for k in 1:n
        i = I[k]
        fname = typeof_constraint(i, robustModel)
        subg = subgradient(xn, data, robustModel.regressionModel, i, fname)
        pin = proj_pi(x->fconstraint(x, data, robustModel.regressionModel, i, fname), subg, xn, i, data, robustModel.regressionModel)
        sum_op = sum_op + ω[k]*pin
        Ln = Ln + ω[k]*((pin-xn)'*(pin-xn))
    end
    un = xn-sum_op
    Ln = Ln/(un'*un)
    return un, Ln
end


# auxilaray function to speed uo proj parallel
@everywhere function auxf(i::Int,
                xn::Array{Float64,1},
                data::Array{Float64,2},
                robustModel::RobustModel,
                ω::Float64)

    name = typeof_constraint(i, robustModel)
    subg = subgradient(xn, data, robustModel.regressionModel, i, name)
    pin::Array{Float64,1} = proj_pi(x->fconstraint(x, data, robustModel.regressionModel, i, name), subg, xn, i, data, robustModel.regressionModel)
    aux::Array{Float64,1} =  ω*vcat(pin, dot((pin-xn),(pin-xn)))

    return aux
end

function proj_In(xn::Array{Float64,1},
                  data::Array{Float64,2},
                  robustModel::RobustModel,
                  I::Array{Int64,1},
                  ω::Array{Float64,1},
                  para::Parallel)

    n::Int = length(I)
    ul::Array{Float64,1} = @parallel (+) for k in 1:n
        auxf(I[k], xn, data, robustModel, ω[k])
    end
    un::Array{Float64,1} = xn - ul[1:end-1]
    Ln::Float64 = ul[end]
    Ln = Ln/dot(un, un)
    return un, Ln
end

proj_In (generic function with 2 methods)

In [ ]:
@btime x_in_inter(x0,Z,robustModel,collect(1:100),Sequential())

In [ ]:
@btime x_in_inter(x0,Z,robustModel,collect(1:100),Parallel())

In [ ]:
xrobust1 = getsolution(mem, ambiguity, 2)
xrobust2 = getsolution(xalg, ambiguity, 2)

In [ ]:
using Ipopt, PyPlot

In [ ]:
if ambiguity == "KLdivergence"
    @time theDRO = KL_opt(Z, ϵ, IpoptSolver(print_level = 2), robustModel.regressionModel)
else ambiguity == "wasserstein"
    @time theDRO = DRO_opt(Z, ϵ, IpoptSolver(print_level = 2), robustModel.regressionModel)
end 
println("DROopt done")
thebis = normal_opt(Z, IpoptSolver(print_level = 2), LinearRegression())
println("MSE done")
X = Z[:,end-1]
Y = Z[:,end]
plot(X, Y, "o")
plot(X, theDRO[1]+theDRO[2]*X, "orange")
plot(X, xrobust1[1]+xrobust1[2]*X, "red")
plot(X, xrobust2[1]+xrobust2[2]*X, "blue")

In [ ]:
plot(dm, color = "black")

In [ ]:
seed = rand(1:100)

In [ ]:
srand(seed)
x1 = initialize(Z, robustModel, robustModel.ambiguitySet)
xalg, yalg, dm1, mem, mini = run_algo(x1, Z, robustModel, optParams, projParams)
srand(seed)
x2 = init_proj(Z, robustModel, projParams)
xalg, yalg, dm2, mem, mini = run_algo(x2, Z, robustModel, optParams, projParams)

In [ ]:
plot(dm1, color = "blue")
plot(dm2, color = "red")

In [ ]:
using BenchmarkTools

In [ ]:
sample = 80
ambiguity = "wasserstein"
robustModel = RobustModel(10, 2, 0.02, ambiguity, LinearRegression())
projParams = ProjParams(Int(1e6), 1e-5, sample, para_proj=Parallel(), para_inter=Parallel())

In [ ]:
srand(2)
@time algo_proj(x0, Z, robustModel, projParams)

In [ ]:
srand(2)
@time algo_proj_bis(x0, Z, robustModel, projParams)

In [ ]:
projParams = ProjParams(Int(1e6), 1e-5, sample, para_proj=Sequential(), para_inter=Sequential())
srand(2)
@time algo_proj(x0, Z, robustModel, projParams)

In [ ]:
# x0 = initialize(Z, robustModel.regressionModel, robustModel.ambiguitySet)
# # ones(size(robustModel.descent_direction))
# Profile.clear()
# @profile run_algo(x0, Z, robustModel, optParams, projParams)

In [ ]:
# ProfileView.view()

In [ ]:
# lintfile("RobustOptimization.jl")

In [ ]:
srand(seed)
x2 = init_proj(Z, robustModel, projParams);
Profile.clear()
@profile run_algo(x2, Z, robustModel, optParams, projParams);
ProfileView.view()

In [10]:
@everywhere function auxf(k::Int, 
                            i::Int, 
                            xn::Array{Float64,1}, 
                            data::Array{Float64,2}, 
                            robustModel::RobustModel, 
                            ω::Float64)
    subg = subgradient(xn, data, robustModel.regressionModel, i, robustModel.fname(i))
    pin::Array{Float64,1} = proj_pi(x->fconstraint(x, data, robustModel.regressionModel, i, robustModel.fname(i)), subg, xn, i, data, robustModel.regressionModel)
    aux::Array{Float64,1} =  ω*vcat(pin, dot((pin-xn),(pin-xn)))
end

In [11]:
@everywhere function proj_In_ter(xn::Array{Float64,1},
                  data::Array{Float64,2},
                  robustModel::RobustModel,
                  I::Array{Int64,1},
                  ω::Array{Float64,1})

    n::Int = length(I)
    ul::Array{Float64,1} = @parallel (+) for k in 1:n
        auxf(k, I[k], xn, data, robustModel, ω[k])::Array{Float64,1}
    end
    un::Array{Float64,1} = xn - ul[1:end-1]
    Ln::Float64 = ul[end]
    Ln = Ln/dot(un, un)
    return un, Ln
end

In [12]:
function proj_In_bis(xn::Array{Float64,1},
                  data::Array{Float64,2},
                  robustModel::RobustModel,
                  I::Array{Int64,1},
                  ω::Array{Float64,1})

    n = length(I)
    pin::Array{Float64,1} = zeros(length(xn))
    sum_op = zeros(length(xn))
    Ln = 0
    for k in 1:n
        i = I[k]
        subg = subgradient(xn, data, robustModel.regressionModel, i, robustModel.fname(i))
        pin = proj_pi(x->fconstraint(x, data, robustModel.regressionModel, i, robustModel.fname(i)), subg, xn, i, data, robustModel.regressionModel)
        sum_op = sum_op + ω[k]*pin
        Ln = Ln + ω[k]*dot((pin-xn),(pin-xn))
    end
    un = xn-sum_op
    Ln = Ln/dot(un, un)
    return un, Ln
end

proj_In_bis (generic function with 1 method)

In [ ]:
I = collect(1:1001)
@btime L, u = proj_In_ter(x0, Z, robustModel, I, ones(length(I))/(length(I)))

In [ ]:
@btime Lb, ub = proj_In_bis(x0, Z, robustModel, I, ones(length(I))/(length(I)))

In [ ]:
Profile.clear()
@profile proj_In_ter(x0, Z, robustModel, I, ones(length(I))/(length(I)))
ProfileView.view()

In [ ]:
L - Lb

In [ ]:
u-ub

In [ ]:
@everywhere function x_in_inter(x, data, robustModel, I)
    aux = @parallel (max) for i in 1:length(I)
        max(0,fconstraint(x, data, robustModel.regressionModel, i, robustModel.fname(i)))
    end
#     aux = max(aux, 0)
    isnan(aux) && (aux = 1.0)
    return aux
end

function x_in_inter_bis(x::Array{Float64,1},
                     data::Array{Float64,2},
                     robustModel::RobustModel,
                     I::Array{Int64,1})

    aux::Float64 = 0.0
    for i in 1:length(I)
        aux = max(aux,fconstraint(x, data, robustModel.regressionModel, i, robustModel.fname(i)))
    end
    isnan(aux) && (aux = 1.0)
    return aux
end

In [ ]:
I = collect(1:10001);

In [ ]:
using BenchmarkTools

In [ ]:
@btime x_in_inter(x0, Z, robustModel, I)

In [ ]:
@btime x_in_inter_bis(x0, Z, robustModel, I)

In [13]:
@everywhere function algo_proj_ter(x0::Array{Float64,1},
                    data::Array{Float64,2},
                    robustModel::RobustModel,
                    projParams::ProjParams)

    dist_mem::Float64 = projParams.precision+1
    xn::Array{Float64,1} = copy(x0)
    iter = 0
    while (iter < Int(round(length(robustModel.I0)/projParams.sample))+1) || ((dist_mem > projParams.precision) && (iter < projParams.ITER_MAX))
        # println("iter = ", iter, " dm = ", dist_mem)
        iter = iter +1
        I = StatsBase.sample(robustModel.I0, projParams.sample, replace=false, ordered=true)
        push!(I, length(robustModel.I0)+1)
        ω = ones(length(I))/(length(I))
        n = length(I)
        # Ln::Float64 = 0
        # aux::Float64 = 0
        res = @parallel (+) for k in 1:n
            i = I[k]
            fname = robustModel.fname(i)
            subg = subgradient(xn, data, robustModel.regressionModel, i, fname)
            pin = proj_pi(x->fconstraint(x, data, robustModel.regressionModel, i, fname), subg, xn, i, data, robustModel.regressionModel)
            aux = fconstraint(xn, data, robustModel.regressionModel, i, robustModel.fname(i))
            vcat(ω[k]*pin, ω[k]*dot((pin-xn),(pin-xn)), aux)
        end
        print("res = ", res)
        un = res[1:end-2]
        Ln = res[end-1]
        dist_mem = res[end]
        un = xn-un
        Ln = Ln/(un'*un)
        if (dist_mem <= 0.0) || (dot(un,un) == 0)
            Ln = 1
            # dist_mem = (xn1-xn)'*(xn1-xn)
            I = StatsBase.sample(robustModel.I0, projParams.sample, replace=false, ordered=true)
            dist_mem = x_in_inter(xn, data, robustModel, I, projParams.para_inter)
        end
        zn = xn - Ln*un
        xn = QR(x0,xn,zn)
    end
    return xn, dist_mem, iter
end

In [ ]:
algo_proj_ter(x0, Z, robustModel, projParams)